In [1]:
from selenium import webdriver
from time import sleep
import os

In [2]:
web = webdriver.Chrome('./chromedriver.exe')
web.implicitly_wait(1)

In [3]:
url_list = dict()
def get_link(driver, count):
    try:
        url = driver.find_element_by_xpath("""//*[@id="{}"]/div[2]/a""".format(count))\
            .get_attribute('href')\
            .split('t_id=')[1]\
            .split('&')[0]
    except Exception:
        return ''
    if url in url_list.keys():
        return ''
    url_list[url] = True
    return url

In [4]:
def scrol(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight); return 10")

In [5]:
def one_day(driver, download_image_count):
    scrol(driver)
    #sleep(5)
    page_ids = []
    for i in range(download_image_count):
        link = get_link(driver, i+1)
        if not link:
            continue
        page_ids.append(link)
    return page_ids

In [6]:
def next_date(now):
    day = get_day(now)
    mon = get_mon(now)
    year = get_year(now)
    if day > 28:
        day = 1
        if mon > 12:
            mon = 1
            year += 1
        else:
            mon += 1
    else:
        day += 1
    return "{year}%02d%02d".format(year=year) % (mon, day)

def get_day(now):
    return int(now[-2:])

def get_mon(now):
    return int(now[-4:6])

def get_year(now):
    return int(now[:4])

In [7]:
def driver_next_day(driver, now):
    try:
        driver.get('https://www.pixiv.net/ranking.php?mode=daily&content=illust&date='+next_date(now))
    except TimeoutException:
        pass
    return next_date(now)

In [8]:
def get_image(driver, illust):
    driver.get('https://www.pixiv.net/member_illust.php?mode=medium&illust_id='+illust)
    

In [9]:
def get_image_url(driver, id):
    id = int(id)
    driver.get('https://www.pixiv.net/member_illust.php?mode=medium&illust_id=%d'%id)
    url, err = get_real_image1()
    if err:
        url, err = get_real_image2()
    if err:
        url, err = get_real_image3()
    if err:
        return False
    return url

In [30]:
all_count = 0
now = "20170101"
for i in range(2000):
    now = driver_next_day(web, now)
    try:
        os.mkdir(f'./Datas/crol/{now}')
    except Exception:
        pass
    links = one_day(web, 35)
    count = 0
    for link in links:
        url = get_image_url(web, link)
        if url is None:
            continue
        web.get(url)
        web.find_element_by_xpath('/html/body/img')
        web.get_screenshot_as_file(f'./Datas/crol/{now}/{count}.jpg')
        count += 1
        all_count += 1
        print(all_count)

KeyboardInterrupt: 

In [ ]:
all_count = 0
now = "20170322"
for i in range(2000):
    try:
        now = driver_next_day(web, now)
        links = one_day(web, 15)
        count = 0
        for link in links:
            try:
                url = get_image_url(web, link)
                print(url)
                if url is None:
                    continue
                web.get(url)
                web.get_screenshot_as_file('./Datas/crol/{now}-{count}.jpg'.format(now=now, count=count))
                count += 1
                all_count += 1
                print(all_count)
            except Exception:
                pass
    except Exception:
        pass

In [75]:
web.implicitly_wait(3)

In [14]:
get_pitures_from_date("20170110")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="12"]/div[2]/a"}
  (Session info: chrome=75.0.3770.90)
  (Driver info: chromedriver=2.34.522940 (1a76f96f66e3ca7b8e57d503b4dd3bccfba87af1),platform=Windows NT 10.0.17134 x86_64)


In [3]:
def get_pitures_from_date(date):
    for i in range(100):
        move_to_date(date)
        urls = get_picture_urls(20)
        for index, url in enumerate(urls):
            if is_exist_piture(url):
                continue
            p_url = piture_url(url)
            if not p_url:
                continue
            print(f"{i}-{date}-{index}-{p_url}")
            save_illust(url, date, index)
        date = next_date(date)

In [4]:
DAILY_RANKING_FORM = "https://www.pixiv.net/ranking.php?mode=daily&content=illust&date="
def move_to_date(date):
    try:
        web.get(DAILY_RANKING_FORM+date)
        return False
    except Exception:
        return True

In [5]:
def get_picture_urls(url_count):
    XPATH = """//*[@id="{}"]/div[2]/a"""
    illust_ids = []
    for i in range(1, url_count):
        eliment = web.find_element_by_xpath(XPATH.format(i))
        raw_link = eliment.get_attribute('href')
        illust_id, err = get_illust_id(raw_link)
        if err:
            continue
        illust_ids.append(illust_id)
    return illust_ids

In [6]:
def get_illust_id(link):
    try:
        illust_id = link.split('t_id=')[1].split('&')[0]
        return illust_id, False
    except Exception:
        return '', False

In [7]:
illust_history = []
def is_exist_piture(lillust_id):
    if lillust_id in illust_history:
        return True
    illust_history.append(lillust_id)
    return False

In [8]:
ILLUST_FORM = 'https://www.pixiv.net/member_illust.php?mode=medium&illust_id='
def piture_url(illust_id):
    web.get(ILLUST_FORM + illust_id)
    url, err = get_real_image1()
    if err:
        url, err = get_real_image2()
    if err:
        url, err = get_real_image3()
    if err:
        return False
    return url
        

In [12]:
def get_real_image1():
    XPATH = '//*[@id="root"]/div[1]/div/div/main/section/div[1]/div/figure/div/div/div/a/img'
    try:
        eliment = web.find_element_by_xpath(XPATH)
        return eliment.get_attribute('src'), False
    except Exception:
        return '', True

def get_real_image2():
    XPATH = '//*[@id="root"]/div[1]/div/div/main/section/div[1]/figure/div[1]/div[2]/div[2]/a/img'
    try:
        eliment = web.find_element_by_xpath(XPATH)
        return eliment.get_attribute('src'), False
    except Exception:
        return '', True

def get_real_image3():
    XPATH = '//*[@id="root"]/div[1]/div/div/main/section/div[1]/div/figure/div/div/div/a/img'
    try:
        eliment = web.find_element_by_xpath(XPATH)
        return eliment.get_attribute('src'), False
    except Exception:
        return '', True


In [10]:
def save_illust(url, date, index):
    try:
        os.mkdir(f'./Datas/crol/mon-{mon}')
        os.mkdir(f'./Datas/crol/{date}')
    except Exception:
        pass
    
    mon = get_mon(date)
    web.get(url)
    web.get_screenshot_as_file(f'./Datas/crol/{date}/{index}.jpg')
    web.get_screenshot_as_file(f'./Datas/crol/mon-{mon}/{index}.jpg')

In [16]:
get_picture_urls(20)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="12"]/div[2]/a"}
  (Session info: chrome=75.0.3770.90)
  (Driver info: chromedriver=2.34.522940 (1a76f96f66e3ca7b8e57d503b4dd3bccfba87af1),platform=Windows NT 10.0.17134 x86_64)
